In [1]:
import pickle
import torch
import numpy as np
import pandas as pd
from PIL import Image

In [2]:
model = pickle.load(open('/content/drive/MyDrive/writer verification/writer_varification_model.sav','rb'))

In [3]:
model

(Sequential(
   (0): Conv2d(1, 5, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
   (1): ReLU()
   (2): Conv2d(5, 10, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
   (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (4): Conv2d(10, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (5): ReLU()
   (6): Conv2d(15, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (8): Flatten(start_dim=1, end_dim=-1)
   (9): Linear(in_features=30000, out_features=5000, bias=True)
   (10): ReLU()
   (11): Linear(in_features=5000, out_features=1000, bias=True)
 ),
 KNeighborsClassifier(n_neighbors=435))

In [4]:
!cp /content/drive/MyDrive/NCVPRIPG_2023_WV_Dataset/semi_test.zip /content/test.zip

In [5]:
!unzip test.zip

Archive:  test.zip
  inflating: semi_test/025e5d31.jpg  
  inflating: semi_test/f2bb12b7.jpg  
  inflating: semi_test/af9f1abe.jpg  
  inflating: semi_test/a4589597.jpg  
  inflating: semi_test/320f6bf9.jpg  
  inflating: semi_test/9d22967f.jpg  
  inflating: semi_test/2323b00c.jpg  
  inflating: semi_test/1d320a7d.jpg  
  inflating: semi_test/dc2f775b.jpg  
  inflating: semi_test/7f8d59e0.jpg  
  inflating: semi_test/0f2bb7bc.jpg  
  inflating: semi_test/ea5005a2.jpg  
  inflating: semi_test/7e02401e.jpg  
  inflating: semi_test/986c5823.jpg  
  inflating: semi_test/ef0814a5.jpg  
  inflating: semi_test/f0ffd208.jpg  
  inflating: semi_test/aa1dd1f1.jpg  
  inflating: semi_test/3f602015.jpg  
  inflating: semi_test/595c2f80.jpg  
  inflating: semi_test/d69cea49.jpg  
  inflating: semi_test/fc22fe8d.jpg  
  inflating: semi_test/51b94946.jpg  
  inflating: semi_test/095aa184.jpg  
  inflating: semi_test/a29e58b8.jpg  
  inflating: semi_test/503a268b.jpg  
  inflating: semi_test/90a82140

In [6]:
!unzip /content/drive/MyDrive/writer\ verification/writer-verification-on-summer-challenge.zip

Archive:  /content/drive/MyDrive/writer verification/writer-verification-on-summer-challenge.zip
  inflating: sample_submission.csv   
  inflating: test.csv                


In [7]:
test_set = pd.read_csv('test.csv')
test_set.head()

,img1_name,img2_name
0,3861e092.jpg,b642bddc.jpg
1,776e0055.jpg,2d4e7d20.jpg
2,4bb7f909.jpg,af1b0fbf.jpg
3,0abee1d8.jpg,1c2641c9.jpg
4,1e18f51e.jpg,9fcf5c47.jpg


In [8]:
output = []
root = 'semi_test/'
l = len(test_set)
model_1 = model[0].to('cuda')
model_2 = model[1]
for i in range(l):
  img_1 = test_set.iloc[i,0]
  img_2 = test_set.iloc[i,1]
  id = img_1+'_'+img_2
  img_1 = Image.open(root+img_1).resize((500,50))
  img_2 = Image.open(root+img_2).resize((500,50))
  X1 = np.array([[np.array(img_1)]])
  X2 = np.array([[np.array(img_2)]])
  with torch.inference_mode(True):
    fea1 = model_1(torch.tensor(X1).to('cuda').float()).cpu().numpy()
    fea2 = model_1(torch.tensor(X2).to('cuda').float()).cpu().numpy()
  diff = np.abs(fea1-fea2)
  proba = model_2.predict_proba(diff)
  output.append((id,proba))
  print(f'\r{100*i/l:.4f}',end='')

99.9906

In [9]:
output

[('3861e092.jpg_b642bddc.jpg', array([[0.05287356, 0.94712644]])),
 ('776e0055.jpg_2d4e7d20.jpg', array([[0.13333333, 0.86666667]])),
 ('4bb7f909.jpg_af1b0fbf.jpg', array([[0.82068966, 0.17931034]])),
 ('0abee1d8.jpg_1c2641c9.jpg', array([[0.16321839, 0.83678161]])),
 ('1e18f51e.jpg_9fcf5c47.jpg', array([[1., 0.]])),
 ('393db1d8.jpg_074ce368.jpg', array([[1., 0.]])),
 ('50a1c3e1.jpg_cb43db93.jpg', array([[0.6045977, 0.3954023]])),
 ('14ced697.jpg_707489be.jpg', array([[0.98850575, 0.01149425]])),
 ('8590ec55.jpg_b225d986.jpg', array([[0.40689655, 0.59310345]])),
 ('4fee32f3.jpg_fe3e95df.jpg', array([[0.0045977, 0.9954023]])),
 ('f3865172.jpg_e802159c.jpg', array([[0.00689655, 0.99310345]])),
 ('fd2c9a5f.jpg_7bb96f8d.jpg', array([[0.05287356, 0.94712644]])),
 ('eca69162.jpg_6fbfad4e.jpg', array([[1., 0.]])),
 ('8f7be832.jpg_f5ab64d2.jpg', array([[0.34482759, 0.65517241]])),
 ('5c3c5b6a.jpg_5623071a.jpg', array([[0.14022989, 0.85977011]])),
 ('71299ac7.jpg_9c5e96fb.jpg', array([[0.949425

In [13]:
id = []
proba = []
for i,p in output:
  id.append(i)
  proba.append(p[0][1])

In [15]:
submision = pd.DataFrame([])

In [17]:
submision['id'] = id
submision['proba'] = proba
submision.head()

,id,proba
0,3861e092.jpg_b642bddc.jpg,0.947126
1,776e0055.jpg_2d4e7d20.jpg,0.866667
2,4bb7f909.jpg_af1b0fbf.jpg,0.179310
3,0abee1d8.jpg_1c2641c9.jpg,0.836782
4,1e18f51e.jpg_9fcf5c47.jpg,0.000000


In [19]:
submision['proba']

0        0.947126
1        0.866667
2        0.179310
3        0.836782
4        0.000000
           ...   
10581    0.958621
10582    0.000000
10583    0.450575
10584    0.988506
10585    0.000000
Name: proba, Length: 10586, dtype: float64

In [20]:
submision.to_csv('Neural Survivor.csv',index = False)